**Index the training dataset into elasticSearch according to their DBOtype**

In [2]:
# %load helper_function.py
import json
import string
import re
from elasticsearch import Elasticsearch

def load_dict_from_json(filename):
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        return data
    except:
        print(f'File \'{filename}\' not found.')
        return None

def save_dict_to_json(doc, filename):
    with open(filename, 'w') as f:
        json.dump(doc, f)


def reset_index(es: Elasticsearch, index_name: str, index_settings) -> None:
    """Clears index"""
    if es.indices.exists(index_name):
        es.indices.delete(index=index_name)

    es.indices.create(index=index_name, body=index_settings)

def preprocess(doc: str) -> str:
    """Preprocesses text to prepare it for feature extraction.

    Args:
        doc: String comprising the unprocessed contents of some email file.

    Returns:
        String comprising the corresponding preprocessed text.
    """
    re_html = re.compile("<[^>]+>")
    doc = re_html.sub(" ", doc)
    #remove pure digits 
    doc=re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b","",doc)
    # Replace punctuation marks (including hyphens) with spaces.
    for c in string.punctuation:
        doc = doc.replace(c, " ")
    return doc.lower()


class Indexer:
    def __init__(self,index: str,index_settings:dict, reset=True):
        self.dictionary = {}     
        self.index = index
        self.index_settings= index_settings
        es = Elasticsearch()
        es.info()
        self.es = es
        if reset:
            self.reset_index()

    
    def preprocess(self, doc: str) -> str:
        """Preprocesses text to prepare it for feature extraction.

    Args:
        doc: String comprising the unprocessed contents of some email file.

    Returns:
        String comprising the corresponding preprocessed text.
    """
        re_html = re.compile("<[^>]+>")
        doc = re_html.sub(" ", doc)
        #remove pure digits 
        doc=re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b","",doc)
        # Replace punctuation marks (including hyphens) with spaces.
        for c in string.punctuation:
            doc = doc.replace(c, " ")
        return doc.lower()

    def reset_index(self) -> None:
        """Clears index"""
        if self.es.indices.exists(self.index):
            self.es.indices.delete(index=self.index)

        self.es.indices.create(index=self.index, body=self.index_settings)
    
    def bulk_index(self,data) -> None:
        """Indexes documents from JSONL file."""
        bulk_data = []
        for item in data:
            bulk_data.append(
                {"index": {"_index": self.index, "_id": item.pop("id")}}
            )
            bulk_data.append(item)
        self.es.bulk(index=self.index, body=bulk_data, refresh=True)
    
    def check_esIndex_count(self)->int:
        self.es.indices.refresh(self.index)
        count = self.es.cat.count(self.index, params={"format": "json"})
        return int(count[0]["count"])

    def check_esIndex_content(self, id:str):
        return self.es.get(index=self.index, id=id)

In [3]:
def prepare_bulk_data(data):
    
    """prepare data  for bulk interface indexing to elasticsearch
    
        returns: bulked data
    """
    dic={}
    i=0
    j=0
    for entry in data:
        i+=1
        if entry['question']==None:
            continue
        if entry['category']=='resource':
            j+=1
            processed_question=preprocess(entry['question'])
            for item in entry['type']:      
                #print(processed_question)
                dic[item]=dic.get(item,"")+processed_question
    collections=[{"id":str(i),"type":DBOtype,"questions":dic[DBOtype]} for i,DBOtype in enumerate(dic)]
    print("----i",i)
    print("----j",j)
    return collections

training_data=load_dict_from_json("../../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json")
prepare_bulk_data(training_data)


----i 17571
----j 9573


In [6]:
INDEX_NAME = "trainning_type_questions"
INDEX_SETTINGS = {
    "mappings": {
        "properties": {
            "type": {
                "type": "text",
                "term_vector": "yes",
                "analyzer": "english",
            },
            "questions": {
                "type": "text",
                "term_vector": "yes",
                "analyzer": "english",
            },

        }
    }
}
training_data=load_dict_from_json("../../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json")
collections=prepare_bulk_data(training_data)
index_trainning_type=Indexer(INDEX_NAME,INDEX_SETTINGS)
index_trainning_type.bulk_index(collections)
len_indexed=index_trainning_type.check_esIndex_count()
print(f'{len_indexed} items are indexed in elasticSearch, index name is {INDEX_NAME}')
#index_trainning_type.check_esIndex_content("300")

----i 17571


C:\Users\junec\AppData\Local\Temp/ipykernel_20572/1590620374.py:80: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if self.es.indices.exists(self.index):
C:\Users\junec\AppData\Local\Temp/ipykernel_20572/1590620374.py:83: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.indices.create(index=self.index, body=self.index_settings)


306


C:\Users\junec\AppData\Local\Temp/ipykernel_20572/1590620374.py:96: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.indices.refresh(self.index)
C:\Users\junec\AppData\Local\Temp/ipykernel_20572/1590620374.py:97: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  count = self.es.cat.count(self.index, params={"format": "json"})


{'_index': 'trainning_type_questions',
 '_type': '_doc',
 '_id': '300',
 '_version': 1,
 '_seq_no': 300,
 '_primary_term': 1,
 'found': True,
 '_source': {'type': 'dbo:Glacier',
  'questions': 'which is  tributary  of  next to lake  of  spitsbergen   '}}

In [5]:
with open("../data/training_types.json", 'w',encoding='utf-8') as f:
  json.dump(collections, f, ensure_ascii=False)
print("training_types.json are saved")